In [46]:
#Library Imports
import pandas as pd
import numpy as np
import pandas as pd
import os


In [57]:
def initialize_portfolio(m):
    return np.ones(m) / m

def calculate_price_relative_vectors(folder_path, tickers):
    """
    Function to calculate the price relative vector for multiple stocks and return them side by side.

    Parameters:
    folder_path (str): The path to the folder containing the stock data.
    tickers (list): List of stock tickers to process.

    Returns:
    pd.DataFrame: A dataframe containing price relative vectors for each stock.
    """

    # Get list of all folder names that start with 'allstocks'
    folder_names = [name for name in os.listdir(folder_path) 
                    if os.path.isdir(os.path.join(folder_path, name)) and name.startswith('allstocks')]

    # Initialize an empty dataframe to store price relative vectors for all tickers
    price_relative_df = pd.DataFrame()

    # Column names expected in the CSV files
    column_names = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Split Factor', 'Earnings', 'Dividends']

    # Loop through each ticker
    for ticker in tickers:
        # Initialize an empty dataframe for the current ticker
        close_price_df = pd.DataFrame(columns=['Ticker', 'Date', 'Close'])

        # Loop through each folder (date)
        for date in folder_names: 
            stock_file = 'table_' + ticker + '.csv'
            file_path = os.path.join(folder_path, date, stock_file)

            # Load Data
            iter_df = pd.read_csv(file_path)
            iter_df.columns = column_names

            # Get Closing Price at End of Day
            last_row = iter_df[['Date', 'Close']].iloc[-1]
            last_row['Ticker'] = ticker

            # Append to the dataframe
            close_price_df = pd.concat([close_price_df, last_row.to_frame().T], ignore_index=True)

        # Convert Date to datetime and set as index
        close_price_df['Date'] = close_price_df['Date'].astype(int)
        close_price_df['Date'] = pd.to_datetime(close_price_df['Date'], format='%Y%m%d')
        close_price_df.set_index('Date', inplace=True)

        # Calculate Price Relative Vector
        close_price_df[ticker] = (close_price_df['Close'] / close_price_df['Close'].shift(1)).fillna(1)

        # Add the ticker's price relative vector to the final dataframe
        if price_relative_df.empty:
            price_relative_df = close_price_df[[ticker]]
        else:
            price_relative_df = price_relative_df.join(close_price_df[[ticker]], how='outer')

    return price_relative_df

def calculate_period_return(b_t, x_t):
    """
    Calculate the return of the portfolio in a single period.
    
    Parameters:
    b_t : array-like
        Portfolio vector at time t.
    x_t : array-like
        Price relative vector at time t.
    
    Returns:
    float : Portfolio period return
    """
    return np.dot(b_t, x_t)

def calculate_cumulative_wealth(b_n_1, price_relative_vectors, S0=1):
    """
    Calculate the cumulative wealth over n periods given a portfolio strategy.
    
    Parameters:
    b_n_1 : list of portfolio vectors
        A list of portfolio vectors for each period.
    price_relative_vectors : array-like
        A matrix of price relative vectors for each period.
    S0 : float, optional
        Initial wealth (default is 1).
    
    Returns:
    float : Cumulative wealth after n periods.
    """
    cumulative_wealth = S0
    for t, x_t in enumerate(price_relative_vectors):
        # Calculate portfolio return for period t
        period_return = calculate_period_return(b_n_1[t], x_t)
        # Update cumulative wealth
        cumulative_wealth *= period_return
    return cumulative_wealth

def calculate_exponential_growth_rate(Sn, n, S0=1):
    """
    Calculate the exponential growth rate (Wn) after n periods.
    
    Parameters:
    Sn : float
        Final cumulative wealth.
    n : int
        Number of periods.
    
    Returns:
    float : Exponential growth rate.
    """
    return (1 / n) * np.log(Sn / S0)


In [60]:
# INITIALIZATION
folder_path = 'C:/Users/nlahanis/OneDrive - Huron Consulting Group/Documents/NYU/Online-Quantitative-Trading-Strategies/Data'
tickers = ['aapl', 'msft', 'goog']  # List of tickers you want to process
b = initialize_portfolio(len(tickers))

# Calculate the relative price vectors for each stock
price_relative_vectors = calculate_price_relative_vectors(folder_path, tickers)

# PORTFOLIO STRATEGY HERE
b_n = np.array([b for _ in range(len(price_relative_vectors))])

# Calculate final cumulative wealth after n periods
Sn = calculate_cumulative_wealth(b_n, np.array(price_relative_vectors))

# Calculate exponential growth rate
Wn = calculate_exponential_growth_rate(Sn, len(price_relative_vectors))

[0.33333333 0.33333333 0.33333333]
[1. 1. 1.]
[0.33333333 0.33333333 0.33333333]
[1.00127229 1.01173272 1.0062021 ]
[0.33333333 0.33333333 0.33333333]
[1.07757601 1.06695098 1.03817505]
[0.33333333 0.33333333 0.33333333]
[1.00617852 1.00468844 0.99280717]
[0.33333333 0.33333333 0.33333333]
[1.04597722 1.05238812 1.04970279]
[0.33333333 0.33333333 0.33333333]
[0.95403567 0.95702328 0.95481928]
[0.33333333 0.33333333 0.33333333]
[1.03149786 1.09114583 1.03199639]
[0.33333333 0.33333333 0.33333333]
[0.98398609 0.95485231 1.        ]
[0.33333333 0.33333333 0.33333333]
[0.96864808 0.98930742 0.97174672]
[0.33333333 0.33333333 0.33333333]
[1.00395256 1.0059914  1.00076394]
[0.33333333 0.33333333 0.33333333]
[0.98888444 0.99371812 0.98540638]
[0.33333333 0.33333333 0.33333333]
[1.09087894 1.07666656 1.07989975]
[0.33333333 0.33333333 0.33333333]
[0.98859295 0.99189094 1.00428721]
[0.33333333 0.33333333 0.33333333]
[1.02819226 1.01061567 1.02091597]
[0.33333333 0.33333333 0.33333333]
[1.006808